In [ ]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

from utils import *

seed=42
PATH = Path('data')
list(PATH.iterdir())

In [ ]:
cols_to_dropcols_to =['is_attributed','ip','day','device','second']
train_filename = 'train_day8_3to16_nextclick_FE.feather'

# Sample run

In [ ]:
val_names = [str(i) for i in list((PATH/'validation').iterdir())]
val_names

In [ ]:
train_filename = 'train_day8_3to16_nextclick_FE.feather'
train_df,y_train = get_train(cols_to_drop,train_filename,10000000)
train_df['Target'] = y_train

del y_train
gc.collect()


In [ ]:
val_names=['data/validation/val2.feather']
val_df,y_val = get_val_by_name(cols_to_drop,val_names)
val_df['Target'] = y_val

del y_val
gc.collect()

In [ ]:
cat_vars = ['hour','minute','channel','os','app']
contin_vars = [for i in list(train_df.columns) if i not in cat_vars or i != 'Target']
n = len(train_df)
for v in cat_vars: train_df[v] = train_df[v].astype('category').cat.as_ordered()

In [ ]:
apply_cats(val_df,train_df)

In [ ]:
for v in contin_vars:
    train_df[v]= train_df[v].astype('float32')
    val_df[v] = val_df[v].astype('float32')

In [ ]:
cat_sz = [(c, len(train_df[c].cat.categories)+1) for c in cat_vars]

emb_szs = [(c, min(50, (c+1)//2)) for _,c in cat_sz]

print(emb_szs)

In [ ]:
emb_dr=?
nn_list=?
nn_dr=?
cycle_len=?
lr=1e-3

In [ ]:
def nll_weight_loss(inp,targ):
    return F.nll_loss(inp,targ,weight=torch.tensor([1,100]))

In [ ]:
df_train, y_train, nas, mapper = proc_df(train_df, 'Target', do_scale=True)
df_val, y_val, nas, mapper = proc_df(val_df, 'Target', do_scale=True, mapper=mapper, na_dict=nas)

md = ColumnarModelData.from_data_frames(str(PATH), df_train,df_val, y_train.astype(np.int8), y_val.astype(np.int8),
                                cat_flds=cat_vars, bs=128,is_reg=False)
model = md.get_learner(emb_szs, n_cont=len(df_train.columns)-len(cat_vars),
           emb_drop=emb_dr, 1, nn_list, nn_dr, use_bn=True)


opt = optim.Adam(model.parameters(), lr)
# m = nn.LogSoftmax()
# loss = nn.NLLLoss()
# target = torch.tensor([1,100])
# output = loss(m(input), target)
# output.backward()

# temp=m.fit(lr,1, metrics=[roc_auc_score],cycle_len=cycle_len)
fit(model,md,1,opt,crit=nll_weight_loss,metrics=[roc_auc_score])

# val_pred,val_targ=m.predict_with_targs()
# if get_oof:
#     oof_train = np.append(oof_train,val_pred)

# val_loss=loss_metric(val_targ,val_pred)

# print('Val RMSE: {}'.format(val_loss))
# cv_losses.append(val_loss)
# print('6 fold results: {}'.format(cv_losses))

# mean_cv_loss = np.mean(cv_losses)
# print("Mean Cross Validation RMSE: {}\n".format(mean_cv_loss))
# return (oof_train,cv_losses) if get_oof else cv_losses